In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math,re
%matplotlib inline

from pyspark.sql.types import *
from pyspark.sql.functions import col, mean, udf, lit, current_timestamp, unix_timestamp, array_contains

from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [14]:
# call spark server
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')  
spark = SparkSession(sc)

In [18]:
# Load data:
PATH_TO_DATA = 'D:\\PSU\\8thLaiOfferProjects\\03032019DSHomework02\\ml-latest-small'
ratings = spark.read.csv(PATH_TO_DATA + '\\ratings.csv', header=True, inferSchema=True)
ratings.cache()
movies = spark.read.csv(PATH_TO_DATA + '\\movies.csv', header=True, inferSchema=True)
movies.cache()
links = spark.read.csv(PATH_TO_DATA + '\\links.csv', header=True, inferSchema=True)
links.cache()

print('number of ratings: %i' % ratings.count())
print('sample of ratings: ')
ratings.show(5)

number of ratings: 100836
sample of ratings: 
+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [16]:
# number of users:
ratings.select('userID').distinct().count()

610

In [19]:
# number of movies:
movies.select('movieID').distinct().count()

9742

In [21]:
# the number of movies rated by users:
ratings.select('movieID').distinct().count()

9724

In [22]:
ratings.createOrReplaceTempView('ratings_view')
movies.createOrReplaceTempView('movies_view')

# list movies not rated before
spark.sql(   # run Spark.SQL query
    '''SELECT DISTINCT movieID 
       FROM movies_view 
       WHERE movieID NOT IN (SELECT DISTINCT movieID FROM ratings_view)
    ''').show()

+-------+
|movieID|
+-------+
|   3456|
|   2939|
|   7020|
|   6668|
|   7792|
|  26085|
|  34482|
|  32160|
|  30892|
|   6849|
|   1076|
|  85565|
|   3338|
|  25855|
|   4194|
|   8765|
|   5721|
|  32371|
+-------+



In [34]:
# list movie genres:
# genres_set = set(movies.select('genres').rdd.flatMap(tuple).flatMap(tuple).collect())
# genres_set

movies.select('genres').rdd.flatMap(tuple).flatMap(tuple).take(3)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 43.0 failed 1 times, most recent failure: Lost task 0.0 in stage 43.0 (TID 1032, localhost, executor driver): java.io.IOException: Cannot run program "python3": CreateProcess error=2, The system cannot find the file specified
	at java.base/java.lang.ProcessBuilder.start(Unknown Source)
	at java.base/java.lang.ProcessBuilder.start(Unknown Source)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:155)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:97)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:108)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(

### 3. Train Recommender System

We use alternating least square (ALS) to train collaborative filtering model. To find the best parameters, grid search and 3-fold cross validation will be used.

In [9]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

als_obj = ALS()

# use grid search to tune parameters
param_grid = ParamGridBuilder()\
    .baseOn({als_obj.coldStartStrategy: 'drop'}) \
    .baseOn({als_obj.userCol: 'userId'}) \
    .baseOn({als_obj.itemCol: 'movieId'}) \
    .baseOn({als_obj.ratingCol: 'rating'}) \
    .baseOn({als_obj.maxIter: 20}) \
    .addGrid(als_obj.regParam, [0.001,0.05,0.1,0.5,1,10,100])\
    .addGrid(als_obj.rank, [6,8,10,12,14])\
    .build()

# define evaluator with mse error
mse_evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

# 3- fold cross validation
cv_setting = CrossValidator(estoimator=als_obj, estimatorParamMaps=params_grid, evaluator=mse_evaluator, numFolds=3)
cv_model = cv_setting.fit(train)

AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?

The combination of λ=0.1 and rank = 8 gives the lowest CV error = 0.762, and test error = 0.773.
The result may slightly change due to the random seed.

### Step 1: Data Wrangling:
#### load rating and movie data:
#### ratings: ratings data consists of about 100,000 ratings given by users to movies. Each row of the DataFrame consists of a userId, movieid, timestamp, together with the rating given by each user.


In [ ]:
spark  # check pyspark is working

In [ ]:
# Load data:
PATH_TO_DATA = 'D:\\PSU\\8thLaiOfferProjects\\03032019DSHomework02\\ml-latest-small'
ratings = spark.read.csv(PATH_TO_DATA + '\\ratings.csv', header=True, inferSchema=True)
ratings.cache()

print('number of ratings: %i' % ratings.count())
print('sample of ratings: ')
ratings.show(5)

In [ ]:
ratings = spark.read.csv('/FileStore/tables/ratings.csv', header=True, inferSchema=True)
ratings.cache()

In [ ]:
ratings.schema

In [ ]:
ratings = ratings.select(
  ratings.userId.cast('int'), ratings.movieId.cast('int'), ratings.rating.cast('float'), (ratings.timestamp.cast('long') * 1000).alias('timestamp'))
ratings.show(4)

In [ ]:
ratings.groupBy('rating').count().toPandas()

In [ ]:
import seaborn as sns

fig, ax = plt.subplots()
sns.barplot(x='rating', y='count', color='#3498db', data=ratings.groupBy('rating').count().toPandas())
plt.savefig('fig_001.png', dpi=500)

In [ ]:
# average rating of movies
ratings.groupBy('MovieId').agg(mean('rating')).toPandas().head()

In [ ]:
# average rating of movies:
ratings.groupBy('MovieId').agg(mean('rating')).toPandas()['avg(rating)'].plot(kind='hist', color='#3498db')
plt.savefig('fig_002.png',dpi=600)

In [ ]:
# average rating from users
ratings.groupBy('userId').agg(mean('rating')).toPandas()['avg(rating)'].plot(kind='hist', color='#3498db')
plt.savefig('fig_003.png',dpi=600)

In [ ]:
# load raw data from csv file
raw_movies = spark.read.csv(PATH_TO_DATA + '\\movies.csv', header=True, inferSchema=True)

print('number of rows %i' % raw_movies.count())
print('raw movie data: ')
raw_movies.show(5, truncate=False)

In [ ]:
# define a UDF to convert the raw genres string to an array of genres and lowercase:
from pyspark.sql.types import ArrayType, StringType
extract_genres = udf(lambda x: x.lower().split('|'), ArrayType(StringType()))

# test:
raw_movies.select('movieId', 'title', extract_genres('genres').alias('genres')).show(5, False)

In [ ]:
# define a UDF to extract the release year from the title, and return the new tiele and year in a struct type:
from pyspark.sql.types import StructType,StructField
import re

def extract_year_fn(title):
    result = re.search('\(\d{4}\)', title)
    try:
        if result:
            group = result.group()
            year = group[1:-1]
            start_pos = result.start()
            title = title[:start_pos-1]
            return (title, year)
        else:
            return (title, 1970)
    except:
        print(title)
    
extract_year = udf(extract_year_fn,\
                  StructType([StructField('title', StringType(), True),\
                             StructField('release_date',StringType(), True)]))

# extract the function
a = 'Jumanji (1995)'
extract_year_fn(a)

In [ ]:
# Function works. Now, create a new DataFrame with the cleaned-up titles, release dates and genres of the movies
movies = raw_movies.select(
  'movieId', extract_year('title').title.alias('title'),\
  extract_year('title').release_date.alias('release_date'),\
  extract_genres('genres').alias('genres'))

print('Cleaned movie data: ')
movies.show(5, truncate=False)

In [ ]:
link_data = spark.read.csv(PATH_TO_DATA + '\\links.csv', header=True, inferSchema=True)

# join movies with links to get TMDB id
movie_data = movies.join(link_data, movies.movieId == link_data.movieId)\
  .select(movies.movieId, movies.title, movies.release_date, movies.genres, link_data.tmdbId)

num_movies = movie_data.count()

print('Cleaned movie data with tmdbId links: ')
movie_data.show(5, False)

In [ ]:
# Display movie posters using TMDb API:

try:
#   from PIL import Image   unnecessary
    import tmdbsimple as tmdb
    from skimage import io

    # replace this variable with your actual TMdb API key
    tmdb.API_KEY = 'b3e7d17ea7101f51be7f876489c5398a'
    print('successfully imported tmdbsimple')

    # base URL for TMDB poster images
    IMAGE_URL = 'http://image.tmdb.org/t/p/w185/'
    movie_id = movie_data.first().tmdbId
    movie_info = tmdb.Movies(movie_id).info()
    movie_poster_url = IMAGE_URL + movie_info['poster_path']
    image = io.imread(movie_poster_url)
    io.imshow(image)
    display(io.show())
#   display(Image(movie_poster_url, width=200))   unnecessary
except Exception:
    print('Cannot import tmdbsimple, no movie posters will be displayed')

### Step 2: Write data in Elasticsearch

run ./bin/elasticsearch from command line to start elasticsearch on windows

In [2]:
from elasticsearch import Elasticsearch

# test ES instance is running
es = Elasticsearch()
es.info(pretty=True)

{'name': 'DESKTOP-03OPD3G',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '2W72Db4xT2a83hRig8uBgQ',
 'version': {'number': '7.0.0',
  'build_flavor': 'unknown',
  'build_type': 'unknown',
  'build_hash': 'b7e28a7',
  'build_date': '2019-04-05T22:55:32.697037Z',
  'build_snapshot': False,
  'lucene_version': '8.0.0',
  'minimum_wire_compatibility_version': '6.7.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [4]:
# es.indices.delete(index='movie_rec')

In [5]:
create_index={
    'settings': {
        'analysis': {
            'analyzer': {
                # This configures the custom analyzer we need to parse vectors 
                # such that the scoring plugin will work correctly.
                'payload_analyzer': {
                    'type': 'custom',
                    'tokenizer': 'whitespace',
                    'filter': 'delimited_payload_filter'
                }
            }
        }
    },
    'mappings': {
        'ratings':{
            # this mapping definition sets up the fields for the rating events
            'properties':{
                'timestamp':{
                    'type':'date'
                },
                'userId': {
                    'type': 'integer'
                },
                'movieId':{
                    'type': 'integer'
                },
                'rating': {
                    'type': 'float'
                }
            }
        },
        'users': {
            # this mapping definition sets up metadata fields for the users
            'properties':{
                'userId':{
                    'type': 'integer'
                },
                '@model': {
                    # this mapping definition sets up the fields for user factor vectors of our model
                    'properties':{
                        'factor':{
                            'type': 'text',
                            'term_vector': 'with_position_offsets_payloads',
                            'analyzer': 'payload_analyzer'
                        },
                        'version': {
                            'type': 'keyword'
                        },
                        'timestamp': {
                            'type': 'date'
                        }
                    }
                }
            }
        },
        'movies': {
            # this mapping definition sets up the metadata fields for the movies
            'properties': {
                'movieId': {
                    'type': 'integer'
                },
                'genres': {
                    'type': 'keyword'
                },
                'release_date': {
                    'type': 'date',
                    'format': 'year'
                },
                '@model': {
                    # this definition mapping sets up fields for movie factor verctors of our model
                    'properties': {
                        'factor': {
                            'type': 'text',
                            'term_vecor': 'with_positions_offsets_payloads',
                            'analyzer': 'payload_analyzer'
                        },
                        'version': {
                            'type': 'keyword'
                        },
                        'timestamp':{
                            'type': 'date'
                        }
                    }
                }
            }
        }
    }
}

#### Load Ratings and movies dataframes into Elasticsearch

In [ ]:
# write rating data
ratings.write.format('es').save('movie_rec/ratings')

# write movie data, specifying the dataframe column to use the id mapping
# each document has an unique id
# if it is not specified, an id will be assigned by Elasticsearch
movie_data.write.format('es').option('es.mapping.id', 'movieId').save('movie_rec/movies')

# check load went okay
print('Movie DF count: %d' % movie_data.count())
print('ES index count: %d' % es.count(index='movie_rec, doc_type='movie)['count'])

# check write went okay
print('Rating DF count: %d' % ratings.count())
print('ES index count: %d' % es.count(index='movie_rec', doc_type='ratings')['count']))

In [ ]:
# test things out by retrieving 3 rating documents from Elasticsearch
es.search(index='movie_rec', doc_type='ratings', q='*', size=3)

In [ ]:
# we can use es to query documents
# e.g. count ratings with timestamp between 2016-01-01 and 2016-02-01
es.count(index='movie_rec', doc_type='ratings', q='timestamp:[2016-01-01 TO 2016-02-01]')

In [ ]:
# test things out by searching for movies containing 'matrix' in the title
es.search(index='movie_rec', doc_type='movies', q='titlematrix', size=3)

#### Step 3: Train a recommender model

In [ ]:
rating_from_es = spark.read.format('es').load('movie_rec/ratings')

# train text split
train, test = ratings_from_es.randomSplit([0.7, 0.3], seed=123)
train.show(5)

In [ ]:
als_obj = ALS()

# use grid search to tune parameters
param_grid = ParamGridBuilder()\
    .basedOn({als_obj.coldStartStartegy: 'drop'})\
    .basedOn({als_obj.userCol: 'userId'})\
    .basedOn({als_obj.itemCol: 'movieId'})\
    .basedOn({als_obj.ratingCol: 'rating'})\
    .basedOn({als_obj.maxIter: 20})\
    .addGrid(als_obj.regParam, [0.001,0.05,0.1,0.5,1,10,100])\
    .addGrid(als_obj.rank, [6,8,10,12,14])\
    .build()

# define evaluator with MSE error
mse_evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

# 3- fold cross validation
cv_setting = CrossValidator(estoimator=als_obj, estimatorParamMaps=params_grid, evaluator=mse_evaluator, numFolds=3)
cv_model = cv_setting.fit(train)

In [ ]:
model = cv_model.bestModel
model.userFactors.show(5)
model.itemFactors.show(5)

In [ ]:
print('parameters of the best model: \n()'.format(cv_setting.getEstimatorParamMaps()[np.argmin(cv_model.avgMetrics)]))
print('---------')
print('cross validation RMSE of the best model: ()'.format(min(cv_model.avgMetrics)))

In [ ]:
a = iter(list(cv_model.avgMetrics[:]))
res = []
for x in [1e-3,0.05,0.1,0.5,1,10,100]:
    for y in [6,8,10,12,14]:
        res.append((x, y, next(a)))
for item in res:
    print('|' + '|'.join(list(map(str, item))) + '|')

In [ ]:
mse_evaluator.evaluate((model.transform(test)))

#### Step 4: Export ALS user and item factor vectors to Elasticsearch

Next step s to export the model factors (shown in the DataFrames above) to Elasticsearch.

Inorder to store the model in the correct format for the index mappings set up, you will need to create some utility functions. These functions will allow you to convert the raw vectors (which are equivalent to a Python list in the factor DataFrame above) to the correct delimited string format. This ensures Elasticsearch will parse the vector field in the model correctly using the delimited token filter custom analyzer you configured earlier.

You'll also create a function to convert a vector and related metadata (such as the Spark model id and a timestamp) into a DataFrame field that matches the model field in the Elasticsearch index mapping.
##### Utility functions for converting factor vectors

In [ ]:
def convert_vector(x):
    '''convert a list or numpy array to delimited token filter format'''
    return ''.join(['%s|%s' % (i, v) for i, v in enumerate(x)])

def reverse_convert(s):
    '''convert a delimited token filter format string back to list format'''
    return [float(f.split('|')[1]) for t in s.split(' ')]

def vector_to_struct(x, verstion, ts):
    '''convert a vector to a SparkSQL Struct with string-format vector and version fields'''
    return (convert_vector(x), version, ts)

vector_struct = udf(vector_to_struct,\
                   StructType([StructField('factor', StringType(), True),\
                              StructField('version', StringTyep(), True),\
                              StructField('timestamp', LongType(), True)]))

In [ ]:
convert_vector([1,2,3])

In [ ]:
# test out the vector conversion function
test_vec = model.userFactors.select('feature').first().features
print(test_vec)
print()
print(convert_vector(test_vec))

#### Convert factor vectors to [factor, version, timestamp] form and write to Elasticsearch

In [ ]:
ver = model.uid
ts = unix_timestamp(current_timestamp())

movie_vectors = model.itemFactors.select('id', vector_struct('feature', lit(ver), ts).alias('@model'))
movie_vectors.select('id', '@model.factor', '@model.version', '@model.timestamp').show(5)

In [ ]:
# write data to ES, use:
# - 'id' as the column to map to ES movie id
# - 'update' write mode for ES, since you want to update new fields only
# - 'append' write mode for Spark
movie_vectors.write.format('es')\
    .option('es.mapping.id', 'id') \
    .option('es.write.operation', 'update') \
    .save('movie_rec/movies', mode='append')

In [ ]:
# write data to ES, use:
# - 'id' as the column to map to ES movie id
# - 'update' write mode for ES, since you want to update new fields only
# - 'append' write mode for Spark
user_vectors.write.format('es')\
    .option('es.mapping.id', 'id') \
    .option('es.write.operation', 'update') \
    .save('movie_rec/users', mode='append')

#### Check the data was written correctly

You can search for a movie to see if the model factor vector was written correctly. You should see a '@model': {'factor': '0|...

In [ ]:
# search for a particular sci-fi movie
es.search(index='movie_rec', doc_type='movies', q='star wars phanton menace', size=1)['hits']['hit'][0]

#### Step 5: Recommend using Elasticsearch

Now that you have loaded your recommendation model into Elasticsearch, you will generate some recommendations. First, you'll need to create a few utility functions for:
* Fetching movie posters from TMDb API (optional)
* Constructing the Elasticsearch function score query to generate recommendations from your factor model
* Given a movie, use this query to find the movies most similar to it
* Given a user, use this query to find the movies with the highest predicted rating to recommend to the user.

In [ ]:
from IPython.display import Image, HTML, display

def get_poster_url(id):
    '''Fectch movie poster image URL from TMDb API given a tmdbId'''
    IMAGE_URL = 'https://image.tmdb.org/t/p/w500'
    try:
        import tmdbsimple as tmdb
        from tmdbsimple import APIKeyError
        try: 
            movie = tmdb.Movies(id).info()
            poster_url = IMAGE_URL + movie['poster_path'] if 'poster_path' in movie and movie['poster_path'] is not None
            return poster_url
        except APIKeyError as ae:
            return 'KEY_ERROR'
    except: Exception as me:
            return 'NA'
        
def fn_query(query_vec, q='*', cosine=False):
    '''
    construct an Elasticsearch function score query.
    
    The query takes as parameters:
        - the field in the candidate document that contains the factor vector
        - the query vector
        - a flag indicating whether to use dot product or consine similarity (normalized dot product) for scores
    
    The query vector passed in will be the user factor vector (if generating recommended movies for a user)
    or movie factor vector (if generating similar movies for a given movie)
    '''
    return {
        'query': {
            'function_score': {
                'query': {
                    'query_string': {
                        'query': q
                    }
                },
                'script_score': {
                    'script': {
                        'inline': 'payload_vector_score',
                        'lang': 'native',
                        'params': {
                            'field': '@model.factor',
                            'vector': query_vec,
                            'cosine': cosine
                        }
                    }
                },
                'boost_mode': 'replace'
            }
        }
    }  # 1:45.73

def get_similar(the_id, q='*', num=10, index='movie_rec', dt='movies'):
    '''
    Given a movie id, execute the recommendation function score query to find similar movies, ranked by cosine similarity
    '''
    response = es.get(index=index, doc_type=dt, id=the_id)
    src = response['_source']
    if '@model' in src and 'factor' in src['@model']:
        raw_vec = src['@model']['factor']
        # our script actually uses the list form for the query vector and handles conversion internally
        query_vec = reverse_convert(raw_vec)
        q = fn_query(query_vec, q=q, cosine=True)
        results = es.search(index, dt, body=q)
        hits = results['hits']['hits']
        return src, hits[1:num+1]

def get_user_recs(the_id, q='*', num=10, index='movie_rec'):
    '''
    Given a user id, execute the recommendation function score query to find top movies, ranked by predicted rating
    '''
    response = es.get(index=index, doc_type='users', id=the_id)
    src = response['_source']
    if '@model' in src and 'factor' in src['@model']:
        raw_vec = src['@model']['factor']
        # our script actually uses the list form for the query vector and handles conversion internally
        query_vec = reverse_convert(raw_vec)
        q = fn_query(query_vec, q=q, cosine=False)
        results = es.search(index, 'movies', body=q)
        hits = results['hits']['hits']
        return src, hits[:num]
        
def get_movies_for_user(the_id, num=10, index='movie_rec'):
    '''Given a user id, get the movie rated by that user, from highest to lowest rated
    '''
    response = es.search(index=index, doc_type='ratings',q='userId:%s' % the_id, size=num, sort=['rating:desc'])
    hits = response['hits']['hits']
    ids = [h['_source']['movieId'] for h in hits]
    movies = es.mget(body=('ids':ids), index=index, doc_type='movies', _source_include=['tmdbId', 'title'])
    movies_hits = movies['docs']
    tmdbids = [h['_source'] for h in movies_hits]
    return tmdbids

def display_user_recs(the_id, q='*', num=10, num_last=10, index='movie_rec'):
    user, recs = get_user_recs(the_id, q, num, index)
    user_movies = get_movies_for_user(the_id, num_last, index)
    # check that posters can be displayed
    first_movie = user_movies[0]
    first_in_url = get_poster_url(first_movie['tmdbId'])
    if first_in_url == 'NA':
        display(HTML('<i>Cannot import tmdbsimple. No movie posters will be displayed!</i>'))
    if first_in_url == 'KEY_ERROR':
        display(HTML('<i>Key error accessing TMDb API. Check your API key. No movie posters will be displayed!</i>'))
    
    # display the movies that this user thas rated highly.
    display(HTML('<h2>Get recommended movies for user id %s</h2>' % the_id))
    display(HTML('<h4>The user has rated the following movies highly:%s</h4>'))
    user_html = '<table border=0>'
    i = 0
    for movie in user_movies:
        movie_in_url = get_poster_url(movie['tmdbId'])
        movie_title = movie['title']
        user_html += '<td><h5>%s</h5><img src=%s width=150></img></td>' % (movie_title, movie_in_url)
        i+=1
        if i % 5 == 0:
            user_html += '</tr><tr>'
    user_html += '</tr><tr>'
    display(HTML(user_html))
    # now display the recommended movies for the user
    display(HTML('<br>'))
    display(HTML('<h2>Recommended movies:</h2>'))
    rec_html = '<table border=0>'
    i=0
    for rec in recs:
        r_in_url = get_poster_url(rec['_source']['tmdbId'])
        r_score = rec['_score']
        r_title = rec['_source']['title']
        rec_html += '<td><h5>%s</h5><img src=%s width=150></img></td><td><td><h5>%2.3f</h5></td>' % (r_title, r_in_url, r_score)
        i += 1
        if i % 5 == 0:
            rec_html += '</tr><tr>'
    rec_html += '</tr></table>'
    display(HTML(rec_html))    
    
def display_similar(the_id, q='*', num=10, index='movie_rec', dt='movies'):
    '''
    Display query movie, together with similar movies and similarity scores, in a table
    '''
    movie, recs = get_similar(the_id, q, num, index, dt)
    q_in_url = get_poster_url(movie['tmdbId'])
    if q_in_url == 'NA':
        display(HTML('<i>Cannot import tmdbsimple. No movie posters will be displayed</i>'))
    if q_in_url == 'KEY_ERROR':
        display(HTML('<i>Key error accessing TMDb API. Check your API key. No movie posters will be displayed!</i>'))
    
    display(HTML('<h2>Get similar movies for:</h2>'))
    sim_html = '<table border=0>'
    i=0
    for rec in recs:
        r_in_url = get_poster_url(rec['_source']['tmdbId'])
        r_score = rec['_score']
        r_title = rec['_source']['title']
        sim_html += '<td><h5>%s<img src=%s width150></img></td></h5>%2.3f</h5></td>' % (r_title, r_in_url, r_score)
        i += 1
        if i % 5 == 0:
            sim_html += '</tr><tr>'
    sim_html += '</tr></table>'
    display(HTML(sim_html))

#### 5(a) Find similar movies for a given movie

To start, you can find movies that are similar to a given movie. This similarity score is computed from the model factor vectors for each movie.

Using this similarity you can show recommendations along the lines of people who liked this movie also liked these.

In [ ]:
display_similar(2628, num=5)   # 46

Now you'll see the power and flexibility that comes from usin a search engine to generate recommendations. Elasticsearch allows you to tweek the movie by recommendation query using any standard search query or filter from free text search through to filters based on time and geo-location.

For example, perhaps you want to remove any movies with 'matrix' in the title from the recommendations. You can do this by simply passing a valid Elasticsearch query string to the recommendation function.

In [ ]:
display_similar(2628, num=5, q='title:(NOT matrix)')

In [ ]:
display_similar(1, num=5, q='genres:children')  # 48

Feel free to check the documentation for the Elasticsearch query_string_query and play with the various queries you can construct by passing in.

#### 5(b) Find movies to recommend to a user

Now, you're ready to generate some movie recommendations, personalized for a specific user.

Given a user, you can recommend movies to that user based on the predicted ratings from your model. In a similar manner to the similar movie recommendation using predicted rating score is computed from the model factor vector for the user and the factor vector for each movie. Recall that the collaborative filtering algorithm that, at a high level, we will recommend movies Liked by other users who Liked the same movies as the given user.

In [ ]:
display_user_recs(12, num=5, num_last=5)

Again, note that since we are using a very small dataset, the results may not be too good. However, we can see that this user seems to like some sci-fi and some comedy films. The recommended movies fall broadly into these categories and seem to be somewhat reasonable.
Next, we can again apply the power of Elasticsearch's filtering capabilities to your recommendation engine. Let's say we only want to recommend movie from the past 5 years. This can be done by adding a date month query to the recommendation function score query.

In [ ]:
display_user_recs(12, num=5, num_last=5, q='release_date:[2012 TO *]')

### 4. Give recommendation

In [ ]:
# es query
# this function is defined in Step 5
display_user_recs(12, num=5, num_last=5, q='release_date:[2012 TO *]')